<h1> Groupe 3 - DashBoard
    
> Script Insertion BD (tfidf) Oracle Notebook

<span class="tocSkip"></span>

> *Authors M2 : Quentin MARCU*

</h1>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Insertion" data-toc-modified-id="Insertion-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Insertion</a></span></li></ul></div>

## Introduction

In [2]:
# Introduction

***Import***

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import unicodedata
from tqdm import tqdm

In [2]:
import cx_Oracle
import os
os.chdir("C:\\Oracle\\instantclient_19_5")

***Path***

In [3]:
path = 'C:/Users/Thoma/Documents/M1_SID/S2/Projet_inter_promo/All_Data/G4/'

***Read files***

In [4]:
tfidf_df = pd.read_csv(path + 'g4_tfidf_ALL_DATA.csv', sep=',', nrows=1000)
label_df = pd.read_csv(path + 'first_1000_rows_prediction_V1.csv', sep=',')

In [5]:
tfidf_df = tfidf_df.rename(
    columns={"tromsø": "troms"}).replace({"tromsø": "troms"})
label_df = label_df.replace({pd.np.nan: None})

***Functions***

In [6]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])


def remove_accents_from_list(input_list):
    for i in range(len(input_list)):
        input_list[i] = remove_accents(input_list[i])
    return input_list

In [7]:
list_word = list(tfidf_df.columns)[2:]

In [8]:
list_word = remove_accents_from_list(list_word)

## Insertion

***Connection***

In [9]:
ORACLE_CONNECT = "ztt1721a/ekigugu6$@(DESCRIPTION=(SOURCE_ROUTE=OFF)(ADDRESS_LIST=(ADDRESS=(PROTOCOL=TCP)(HOST=telline.univ-tlse3.fr)(PORT=1521)))(CONNECT_DATA=(SID=etupre)))"
mydb = cx_Oracle.connect(ORACLE_CONNECT)

In [10]:
mycursor = mydb.cursor()
mycursor.arraysize = len(list_word)+10
sql = "INSERT INTO Word (id_word, word) VALUES (:1, :2)"
i = 1
for word in tqdm(list_word):
    mycursor.execute(sql, (i, word))
    i += 1
mydb.commit()

100%|██████████| 16428/16428 [00:31<00:00, 517.60it/s]


In [11]:
mycursor = mydb.cursor()
mycursor.arraysize = label_df.shape[0]+10
sql = "INSERT INTO Document (id_doc, ATMOSPHERE, BAGGAGE, CABIN_CREW, COMFORT, EMPTY, FOOD, NOT_FLIGHT, PRICE, PUNCTUALITY) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)"

for i in tqdm(range(0, label_df.shape[0])):
    line = list(label_df.iloc[i, :])
    val = (i, line[1], line[2], line[3], line[4],
           line[5], line[6], line[7], line[8], line[9])
    mycursor.execute(sql, val)
mydb.commit()

100%|██████████| 7862/7862 [00:16<00:00, 464.03it/s]


In [15]:
mycursor = mydb.cursor()
mycursor.arraysize = 100000
sql = "INSERT INTO Appear (id_doc,id_word,tfidf) VALUES (:1, :2, :3)"

for i in tqdm(range(0, tfidf_df.shape[0])):
    line = list(tfidf_df.iloc[i, 2:])
    for j in range(0, len(line)):
        if line[j] != 0:
            val = (i, j, line[j])
            try:
                mycursor.execute(sql, val)
            except:
                pass
mydb.commit()

 72%|███████▏  | 721/1000 [01:10<00:46,  6.06it/s]

(720, 0, 0.09428315818222212)


 90%|█████████ | 903/1000 [01:33<00:09, 10.53it/s]

(902, 0, 0.1975138058270804)


100%|██████████| 1000/1000 [01:44<00:00,  9.53it/s]
